# 加载w2v 和 sif

In [1]:
doc = '''1937年7月7日夜，枪声划破宛平城。日军炮轰宛平，悍然发动震惊中外的“七七事变”。自此，中国人民抗日战争全面爆发。

在中国共产党倡导建立的抗日民族统一战线的旗帜下，中华儿女以血肉之躯和不屈意志书写了拯救民族危亡的壮丽篇章，取得了中国自近代以来抗击外敌入侵的第一次全面胜利。

“前事不忘，后事之师。”十八大以来，习近平总书记在多个重要场合就中国人民抗日战争作出重要论述，强调弘扬伟大的抗战精神，为民族奋进凝聚起磅礴伟力。



中国人民抗日战争是中华民族历史发展进程中饱经沧桑的章节，是中华民族惊天动地的壮举和伟业，在世界反法西斯战争中具有重要地位。

2014年2月27日，第十二届全国人大常委会第七次会议通过决定，将每年的9月3日设立为中国人民抗日战争胜利纪念日，将12月13日设立为南京大屠杀死难者国家公祭日。

上海社会科学院历史研究所研究员李志茗表示：“这是以国家立法形式确立中国人民抗日战争胜利纪念日和南京大屠杀死难者国家公祭日，将这两个重大日子的纪念活动制度化、规范化。纪念日和公祭日在2014年正式确定后，当年即开展隆重纪念活动。”

这是以习近平同志为核心的党中央高度重视“抗战精神”的重要体现。与此同时，十八大以来，习近平在多个场合发表了一系列关于中国人民抗日战争的重要讲话，高度评价抗日战争的伟大胜利。'''

In [ ]:
from gensim.models import Word2Vec,LdaModel, KeyedVectors
import pickle
import numpy as np 
import re
import jieba
from scipy.spatial.distance import cosine
model_path = os.path.join(os.path.abspath('./'),'word2vector_Model','word2vec.kv')
model = KeyedVectors.load(model_path,mmap='r')
word_sifs_path = os.path.join(os.path.abspath('./'),'data','word_sifs.plk')
word_sifs =pickle.load(open(word_sifs_path,'rb'))

## 分句，分词

In [ ]:
sens = list(filter(lambda x : str.strip(x) , re.split('[。\n]',doc)))
sens = list(map(lambda x:''.join(re.findall(r'\w+',x)),sens))
result = []
for a in sens:
    if a: result.append(list(filter(lambda x: x in model,list(jieba.cut(a)))))

## 加权平均

In [ ]:
embs = []
for s in result:
    emb = np.zeros(100)
    for w in s:
        emb = emb + model[w] * word_sifs[w]
    embs.append(emb / len(s) )
embs = np.asarray(embs)

## 去除第一阶主成分

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
svd.fit(embs)
pc = svd.components_

In [ ]:
embs_pcremove = embs - embs.dot(pc.transpose()) * pc

## 测试

In [ ]:
for index, s in enumerate(result):
    print( index, ''.join(s))

In [ ]:
cosine(embs_pcremove[0],embs_pcremove[1])